# Canada Vigilance Adverse Reaction Online Database

https://www.canada.ca/en/health-canada/services/drugs-health-products/medeffect-canada/adverse-reaction-database/canada-vigilance-adverse-reaction-online-database-data-structures.html

The Canada Vigilance Adverse Reaction Online Database contains information about suspected adverse reactions (also known as side effects) to health products.

Adverse reaction reports are submitted by:

consumers and health professionals, who submit reports voluntarily
manufacturers and distributors (also known as market authorization holders), who are required to submit reports according to the Food and Drugs Act

This database includes data from 1965 to 2025-06-30.

Information about vaccines used for immunization has been included in the database since January 1, 2011. Information about human blood and blood components has been included in the database since September 1, 2015.

The database should not be used on its own to evaluate a health product's safety profile as It does not provide conclusive information on the safety of health products. It should not be used as a substitute for medical advice. If you have a medical concern, consult a qualified health professional. For more information, consult our Interpretation of suspected adverse reaction data page.

For general questions about the content of the Adverse Reaction Online Database or for technical support, please contact the Canada Vigilance Program.

En esta notebook solo salvo los archivos limpios de cephalosporinas. Ver la priemera seccion `Drug Product Ingredients` para ver como se hizo el filtrado. Ver la seccion de `Report Drug` para ver como se obtuvieron los reportes relacionados a cephalosporinas.

En `Drug Product Ingredients` se obtuvieron los nombres de medicamentos que usan cephalosporinas (tuvieran *ceph* o *cef* como ingrediente activo) y luego se usaron para filtrar los reportes en `Report Drug`. Finalmente, se usaron los `REPORT_ID` resultantes para filtrar las demas tablas y obtener solo la informacion relacionada a cephalosporinas.

In [1]:
import polars as pl

PATH = "data/Canada Vigilance Adverse Reaction Online Database/"
OUTPUT = "data/raw/Canada Vigilance Adverse Reaction Online Database/"

# create output directory if it doesn't exist
import os
os.makedirs(OUTPUT, exist_ok=True)

## Drug Product Ingredients

In [6]:
dpi = pl.read_csv(
    PATH + "drug_product_ingredients.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
dpi

column_1,column_2,column_3,column_4,column_5
i64,i64,str,i64,str
29599010225,29599,"""BENZYDAMINE HYDROCHLORIDE/DICH…",10225,"""lidocaine hydrochloride"""
31212010225,31212,"""XYLOCAINE TOPICAL""",10225,"""lidocaine hydrochloride"""
54995010225,54995,"""BECLOMETHASONE DIPROPIONATE/LI…",10225,"""lidocaine hydrochloride"""
55044010225,55044,"""ALUMINUM HYDROXIDE/DIPHENHYDRA…",10225,"""lidocaine hydrochloride"""
58165010225,58165,"""LIDOCAINE HYDROCHLORIDE/NOREPI…",10225,"""lidocaine hydrochloride"""
…,…,…,…,…
3322005010254,33220,"""DIPHTHERIA TOXOID (+) PERTUSSI…",5010254,"""diphtheria toxoid (crm197)"""
3334505010254,33345,"""DTAP-IPV-HIB""",5010254,"""diphtheria toxoid (crm197)"""
3322705008968,33227,"""REACTINE UNSPECIFIED""",5008968,"""cetirizine hcl"""


In [7]:
dpi = dpi.rename({
    "column_1":  "DRUG_PRODUCT_INGREDIENT_ID",
    "column_2":  "DRUG_PRODUCT_ID",
    "column_3":  "DRUGNAME",
    "column_4":  "ACTIVE_INGREDIENT_ID",
    "column_5":  "ACTIVE_INGREDIENT_NAME"
})

In [10]:
dpi

DRUG_PRODUCT_INGREDIENT_ID,DRUG_PRODUCT_ID,DRUGNAME,ACTIVE_INGREDIENT_ID,ACTIVE_INGREDIENT_NAME
i64,i64,str,i64,str
29599010225,29599,"""BENZYDAMINE HYDROCHLORIDE/DICH…",10225,"""lidocaine hydrochloride"""
31212010225,31212,"""XYLOCAINE TOPICAL""",10225,"""lidocaine hydrochloride"""
54995010225,54995,"""BECLOMETHASONE DIPROPIONATE/LI…",10225,"""lidocaine hydrochloride"""
55044010225,55044,"""ALUMINUM HYDROXIDE/DIPHENHYDRA…",10225,"""lidocaine hydrochloride"""
58165010225,58165,"""LIDOCAINE HYDROCHLORIDE/NOREPI…",10225,"""lidocaine hydrochloride"""
…,…,…,…,…
3322005010254,33220,"""DIPHTHERIA TOXOID (+) PERTUSSI…",5010254,"""diphtheria toxoid (crm197)"""
3334505010254,33345,"""DTAP-IPV-HIB""",5010254,"""diphtheria toxoid (crm197)"""
3322705008968,33227,"""REACTINE UNSPECIFIED""",5008968,"""cetirizine hcl"""


In [11]:
dpi.write_parquet(OUTPUT + "drug_product_ingredients.parquet")

In [2]:
# read dpi
dpi = pl.read_parquet(OUTPUT + "drug_product_ingredients.parquet")

In [3]:
ceph = dpi.filter(
    pl.col("ACTIVE_INGREDIENT_NAME")
      .cast(pl.Utf8)
      .str.contains(r"(?i)\bcef|\bceph")   # (?i) = case-insensitive
)
ceph

DRUG_PRODUCT_INGREDIENT_ID,DRUG_PRODUCT_ID,DRUGNAME,ACTIVE_INGREDIENT_ID,ACTIVE_INGREDIENT_NAME
i64,i64,str,i64,str
2886208326,28862,"""CEFUROXIME AXETIL/PROBENECID""",8326,"""cefuroxime axetil"""
29745011102,29745,"""KEFLEX /UNK/""",11102,"""cephalexin monohydrate"""
7775011118,7775,"""CEFUROXIME FOR INJECTION, USP""",11118,"""cefuroxime"""
21878011118,21878,"""RATIO-CEFUROXIME""",11118,"""cefuroxime"""
22959011118,22959,"""KEFUROX""",11118,"""cefuroxime"""
…,…,…,…,…
167805014176,1678,"""NOVO-LEXIN""",5014176,"""cefalexin"""
195806012648,1958,"""NOVO-LEXIN""",6012648,"""methylol cefalexin lysinate"""
3363505014176,33635,"""CEPHALEXIN 500mg""",5014176,"""cefalexin"""


In [12]:
drugnames = ceph['DRUGNAME']
ingredient = ceph['ACTIVE_INGREDIENT_NAME']
#sort by drugnames
drugnames, ingredient = zip(*sorted(zip(drugnames, ingredient)))

for dn, ing in zip(drugnames, ingredient):
    print(f"{dn}: {ing}")

ANCEF: cefazolin
ANCEF (CEFAZOLIN SODIUM): cefazolin
ANCEF INJ PWS 10GM/VIAL USP: cefazolin
ANCEF INJ PWS 1GM/VIAL USP: cefazolin
APO-CEFACLOR: cefaclor
APO-CEFACLOR: cefaclor
APO-CEFACLOR: cefaclor monohydrate
APO-CEFACLOR: cefaclor monohydrate
APO-CEFADROXIL: cefadroxil
APO-CEFADROXIL CAPSULES: cefadroxil
APO-CEFEPIME: cefepime
APO-CEFEPIME: cefepime dihydrochloride monohydrate
APO-CEFOXITIN INJECTABLE: cefoxitin
APO-CEFPROZIL: cefprozil
APO-CEFPROZIL: cefprozil
APO-CEFPROZIL: cefprozil
APO-CEFPROZIL: cefprozil
APO-CEFUROXIME: cefuroxime
APO-CEFUROXIME: cefuroxime
APO-CEFUROXIME: cefuroxime
APO-CEFUROXIME: cefuroxime salt not specified
APO-CEFUROXIME: cefuroxime salt not specified
APO-CEFUROXIME: cefuroxime salt not specified
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin
APO-CEPHALEX: cefalexin sodium
APO-CEPHALEX: c

## Drug Products

In [ ]:
dp = pl.read_csv(
    PATH + "drug_products.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
dp

column_1,column_2
i64,str
1,"""VITAMIN B12 TAB 100MCG"""
2,"""VITAMIN B12 TAB 250MCG"""
3,"""TUBERCULIN PPD (MANTOUX)"""
4,"""TUBERSOL"""
5,"""APO TOLBUTAMIDE TAB 500MG"""
…,…
38699,"""LOXAPINE"""
38704,"""LOXAPINE MYLAN"""
38712,"""LOZENGE"""


In [ ]:
dp.filter(pl.col("column_2").str.starts_with("CEF"))

column_1,column_2
i64,str
1338,"""CEFTIN TAB 250MG"""
1339,"""CEFTIN TAB 500MG"""
1454,"""CEFTAZIDIME FOR INJECTION, USP"""
1455,"""CEFTAZIDIME FOR INJECTION, USP"""
4281,"""CEFTIN - TAB 250MG"""
…,…
59521,"""CEFATRIZINE"""
60138,"""CEFTAZIDIME PENTAHYDRATE"""
51983,"""CEFALEXIN"""


In [ ]:
dp = dp.rename({
    "column_1":  "DRUG_PRODUCT_ID",
    "column_2":  "DRUGNAME",
})
dp.write_parquet(OUTPUT + "drug_product.parquet")

## Outcomes

In [ ]:
outcomes = pl.read_csv(
    PATH + "outcome_lx.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
outcomes

column_1,column_2,column_3,column_4
i64,i64,str,str
1906,6,"""Unknown""","""Inconnue"""
1907,7,"""Recovered/resolved""","""Guérison"""
1908,8,"""Recovering/resolving""","""Guérison en cours"""
1909,9,"""Not recovered/not resolved""","""Non rétabli/Non résolu"""
1910,10,"""Recovered/resolved with sequel…","""Guérison avec séquelles"""
1911,11,"""Fatal""","""Fatale"""


In [ ]:
outcomes = outcomes.rename({
    "column_1":  "OUTCOME_LX_ID",
    "column_2":  "OUTCOME_CODE",
    "column_3":  "OUTCOME_EN",
    "column_4":  "OUTCOME_FR"
})
outcomes.write_parquet(OUTPUT + "outcomes.parquet")

## Reactions

In [15]:
reactions = pl.read_csv(
    PATH + "reactions.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
reactions

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10
i64,i64,str,str,str,str,str,str,str,str
9701,97,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
9801,98,"""""","""""","""""","""Completed suicide""","""Suicide""","""Psychiatric disorders""","""Affections psychiatriques""","""v.28.0"""
9901,99,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
10001,100,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
10101,101,"""""","""""","""""","""Dizziness""","""Sensation vertigineuse""","""Nervous system disorders""","""Affections du système nerveux""","""v.28.0"""
…,…,…,…,…,…,…,…,…,…
90803319501,908033195,"""""","""""","""""","""Drug ineffective""","""Inefficacité médicamenteuse""","""General disorders and administ…","""Troubles généraux et anomalies…","""v.28.0"""
90803320103,908033201,"""""","""""","""""","""Stomatitis""","""Stomatite""","""Gastrointestinal disorders""","""Affections gastro-intestinales""","""v.28.0"""
90803320102,908033201,"""""","""""","""""","""Abdominal distension""","""Distension abdominale""","""Gastrointestinal disorders""","""Affections gastro-intestinales""","""v.28.0"""


In [ ]:
reactions = reactions.rename({
    "column_1":  "REACTION_ID",
    "column_2":  "REPORT_ID",
    "column_3":  "DURATION",
    "column_4":  "DURATION_UNIT_ENG",
    "column_5":  "DURATION_UNIT_FR",
    "column_6":  "PT_NAME_ENG",
    "column_7":  "PT_NAME_FR",
    "column_8":  "SOC_NAME_ENG",
    "column_9":  "SOC_NAME_FR",
    "column_10":  "MEDDRA_VERSION"
})

reactions.write_parquet(OUTPUT + "reactions.parquet")
reactions

REACTION_ID,REPORT_ID,DURATION,DURATION_UNIT_ENG,DURATION_UNIT_FR,PT_NAME_ENG,PT_NAME_FR,SOC_NAME_ENG,SOC_NAME_FR,MEDDRA_VERSION
i64,i64,str,str,str,str,str,str,str,str
9701,97,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
9801,98,"""""","""""","""""","""Completed suicide""","""Suicide""","""Psychiatric disorders""","""Affections psychiatriques""","""v.28.0"""
9901,99,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
10001,100,"""""","""""","""""","""Accidental overdose""","""Surdosage accidentel""","""Injury, poisoning and procedur…","""Lésions, intoxications et comp…","""v.28.0"""
10101,101,"""""","""""","""""","""Dizziness""","""Sensation vertigineuse""","""Nervous system disorders""","""Affections du système nerveux""","""v.28.0"""
…,…,…,…,…,…,…,…,…,…
90803319501,908033195,"""""","""""","""""","""Drug ineffective""","""Inefficacité médicamenteuse""","""General disorders and administ…","""Troubles généraux et anomalies…","""v.28.0"""
90803320103,908033201,"""""","""""","""""","""Stomatitis""","""Stomatite""","""Gastrointestinal disorders""","""Affections gastro-intestinales""","""v.28.0"""
90803320102,908033201,"""""","""""","""""","""Abdominal distension""","""Distension abdominale""","""Gastrointestinal disorders""","""Affections gastro-intestinales""","""v.28.0"""


## Report Drug Indications

In [ ]:
rdi = pl.read_csv(
    PATH + "report_drug_indication.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
rdi

column_1,column_2,column_3,column_4,column_5,column_6
i64,i64,i64,str,str,str
3973977,300792,9496,"""ADVIL COLD & SINUS LIQUI-GELS""","""Nasopharyngitis""","""Rhinopharyngite"""
3893241,300764,31952,"""CODEINE""","""Product used for unknown indic…","""Produit utilisé pour une indic…"
3974079,300440,9589,"""FORTEO""","""Osteoporosis""","""Ostéoporose"""
3977017,300944,29039,"""MIRTAZAPINE""","""Insomnia""","""Insomnie"""
3983680,301511,10737,"""CHAMPIX""","""Smoking cessation therapy""","""Traitement pour arrêter de fum…"
…,…,…,…,…,…
5295423,908147986,16362,"""QULIPTA""","""Migraine""","""Migraine"""
4956401,907353481,24811,"""ROPIVACAINE HYDROCHLORIDE""","""Product used for unknown indic…","""Produit utilisé pour une indic…"
5213285,908120465,52875,"""DICLOFENAC POTASSIUM""","""Product used for unknown indic…","""Produit utilisé pour une indic…"


In [ ]:
rdi = rdi.rename({
    "column_1":  "REPORT_DRUG_ID",
    "column_2":  "REPORT_ID",
    "column_3":  "DRUG_PRODUCT_ID",
    "column_4":  "DRUGNAME",
    "column_5":  "INDICATION_NAME_ENG",
    "column_6":  "INDICATION_NAME_FR"
})
rdi.write_parquet(OUTPUT + "report_drug_indication.parquet")

## Report Drug

In [23]:
report_drug = pl.read_csv(
    PATH + "report_drug.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
report_drug

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
265,136627,30441,"""MINOCYCLINE""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
266,136627,33686,"""PULMICORT""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
267,166980,33571,"""VASOTEC""","""Concomitant""","""Concomitant""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
269,13557,34527,"""HYDROCORTISONE""","""Suspect""","""Suspect""","""Rectal""","""Rectale""","""100""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""7""","""Days""","""Jour(s)""","""Enema""","""Lavement"""
271,13557,32824,"""PREDNISONE""","""Suspect""","""Suspect""","""Oral""","""Oral""","""40""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""14""","""Days""","""Jour(s)""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4399335,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Oral""","""Oral""","""5""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399342,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Unknown""","""Inconnu""","""4""","""Milligram""","""Milligramme""","""1""","""1""","""Days""","""Jour(s)""","""1 every 1 Days""","""1 chaque 1 Jour(s)""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399348,907360553,34916,"""OTEZLA""","""Suspect""","""Suspect""","""Oral""","""Oral""","""3""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [24]:
report_drug = report_drug.rename({
    "column_1":  "REPORT_DRUG_ID",
    "column_2":  "REPORT_ID",
    "column_3":  "DRUG_PRODUCT_ID",
    "column_4":  "DRUGNAME",
    "column_5":  "DRUGINVOLV_ENG",
    "column_6":  "DRUGINVOLV_FR",
    "column_7":  "ROUTEADMIN_ENG",
    "column_8":  "ROUTEADMIN_FR",
    "column_9":  "UNIT_DOSE_QTY",
    "column_10": "DOSE_UNIT_ENG",
    "column_11": "DOSE_UNIT_FR",
    "column_12": "FREQUENCY",
    "column_13": "FREQ_TIME",
    "column_14": "FREQUENCY_TIME_ENG",
    "column_15": "FREQUENCY_TIME_FR",
    "column_16": "FREQ_TIME_UNIT_ENG",
    "column_17": "FREQ_TIME_UNIT_FR",
    "column_18": "THERAPY_DURATION",
    "column_19": "THERAPY_DURATION_UNIT_ENG",
    "column_20": "THERAPY_DURATION_UNIT_FR",
    "column_21": "DOSAGEFORM_ENG",
    "column_22": "DOSAGEFORM_FR"
})



In [25]:
report_drug

REPORT_DRUG_ID,REPORT_ID,DRUG_PRODUCT_ID,DRUGNAME,DRUGINVOLV_ENG,DRUGINVOLV_FR,ROUTEADMIN_ENG,ROUTEADMIN_FR,UNIT_DOSE_QTY,DOSE_UNIT_ENG,DOSE_UNIT_FR,FREQUENCY,FREQ_TIME,FREQUENCY_TIME_ENG,FREQUENCY_TIME_FR,FREQ_TIME_UNIT_ENG,FREQ_TIME_UNIT_FR,THERAPY_DURATION,THERAPY_DURATION_UNIT_ENG,THERAPY_DURATION_UNIT_FR,DOSAGEFORM_ENG,DOSAGEFORM_FR
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
265,136627,30441,"""MINOCYCLINE""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
266,136627,33686,"""PULMICORT""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
267,166980,33571,"""VASOTEC""","""Concomitant""","""Concomitant""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
269,13557,34527,"""HYDROCORTISONE""","""Suspect""","""Suspect""","""Rectal""","""Rectale""","""100""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""7""","""Days""","""Jour(s)""","""Enema""","""Lavement"""
271,13557,32824,"""PREDNISONE""","""Suspect""","""Suspect""","""Oral""","""Oral""","""40""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""14""","""Days""","""Jour(s)""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4399335,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Oral""","""Oral""","""5""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399342,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Unknown""","""Inconnu""","""4""","""Milligram""","""Milligramme""","""1""","""1""","""Days""","""Jour(s)""","""1 every 1 Days""","""1 chaque 1 Jour(s)""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399348,907360553,34916,"""OTEZLA""","""Suspect""","""Suspect""","""Oral""","""Oral""","""3""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [ ]:
ceph 

DRUG_PRODUCT_INGREDIENT_ID,DRUG_PRODUCT_ID,DRUGNAME,ACTIVE_INGREDIENT_ID,ACTIVE_INGREDIENT_NAME
i64,i64,str,i64,str
2886208326,28862,"""CEFUROXIME AXETIL/PROBENECID""",8326,"""cefuroxime axetil"""
29745011102,29745,"""KEFLEX /UNK/""",11102,"""cephalexin monohydrate"""
7775011118,7775,"""CEFUROXIME FOR INJECTION, USP""",11118,"""cefuroxime"""
21878011118,21878,"""RATIO-CEFUROXIME""",11118,"""cefuroxime"""
22959011118,22959,"""KEFUROX""",11118,"""cefuroxime"""
…,…,…,…,…
167805014176,1678,"""NOVO-LEXIN""",5014176,"""cefalexin"""
195806012648,1958,"""NOVO-LEXIN""",6012648,"""methylol cefalexin lysinate"""
3363505014176,33635,"""CEPHALEXIN 500mg""",5014176,"""cefalexin"""


In [31]:
ceph.select("ACTIVE_INGREDIENT_NAME").unique().sort("ACTIVE_INGREDIENT_NAME")

# print them
active_ingredients = ceph.select("ACTIVE_INGREDIENT_NAME").unique().sort("ACTIVE_INGREDIENT_NAME")
for ai in active_ingredients["ACTIVE_INGREDIENT_NAME"]:
    print(ai)

cefaclor
cefaclor monohydrate
cefadroxil
cefadroxil monohydrate
cefalexin
cefalexin hydrochloride
cefalexin monohydrate
cefalexin sodium
cefaloridine
cefalotin
cefamandole nafate
cefapirin
cefatrizine
cefazolin
cefazolin benzathine
cefazolin sodium
cefazolin, sodium
cefazoline benzathine
cefcapene
cefcapene pivoxil
cefcapene pivoxil hcl
cefcapene pivoxil hydrochloride
cefcapene pivoxil hydrochloride hydrate
cefdinir
cefepim
cefepime
cefepime dihydrochloride monohydrate
cefepime hcl
cefepime hydrochloride
cefepime hydrochloride monohydrate
cefepime, hydrochloride, monohydrate
cefixima
cefixime
cefixime trihydrate
cefmetazole sodium
cefodizime
cefodizime disodium
cefodizime sodium
cefoperazone sodium
cefotaxime
cefotaxime sodique
cefotaxime sodium
cefotetan
cefotetan disodium
cefotiam
cefotiam hexetil hydrochloride
cefotiam hydrochloride
cefoxitin
cefoxitin sodium
cefpodoxim
cefpodoxime
cefpodoxime proxetil
cefprozil
cefprozil monohydrate
cefradine
cefsulodin
cefsulodin sodium
ceftarolin

In [21]:
interest_drugnames = ceph.select("DRUG_PRODUCT_ID").unique().sort("DRUG_PRODUCT_ID")
interest_drugnames

DRUG_PRODUCT_ID
i64
208
394
451
858
925
…
58966
59521
60138


In [37]:
# flter to see where the drugname is KEFUROX
ceph.filter(pl.col("DRUGNAME").str.contains("KEFUROX"))

DRUG_PRODUCT_INGREDIENT_ID,DRUG_PRODUCT_ID,DRUGNAME,ACTIVE_INGREDIENT_ID,ACTIVE_INGREDIENT_NAME
i64,i64,str,i64,str
22959011118,22959,"""KEFUROX""",11118,"""cefuroxime"""
2295905011537,22959,"""KEFUROX""",5011537,"""cefuroxime salt not specified"""


In [45]:
# filter to see the report id 164
report_drug.filter(pl.col("REPORT_ID") == 173)

REPORT_DRUG_ID,REPORT_ID,DRUG_PRODUCT_ID,DRUGNAME,DRUGINVOLV_ENG,DRUGINVOLV_FR,ROUTEADMIN_ENG,ROUTEADMIN_FR,UNIT_DOSE_QTY,DOSE_UNIT_ENG,DOSE_UNIT_FR,FREQUENCY,FREQ_TIME,FREQUENCY_TIME_ENG,FREQUENCY_TIME_FR,FREQ_TIME_UNIT_ENG,FREQ_TIME_UNIT_FR,THERAPY_DURATION,THERAPY_DURATION_UNIT_ENG,THERAPY_DURATION_UNIT_FR,DOSAGEFORM_ENG,DOSAGEFORM_FR
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
4282812,173,24348,"""GARAMYCIN""","""Concomitant""","""Concomitant""","""Intravenous (not otherwise spe…","""Intraveineuse (sans autre préc…","""80""","""Milligram""","""Milligramme""","""3""","""1""","""Days""","""Jour(s)""","""3 every 1 Days""","""3 chaque 1 Jour(s)""","""""","""""","""""","""SOLUTION INTRAMUSCULAR""","""SOLUTION POUR ADMINISTRATION I…"
4282811,173,32068,"""ERYTHROMYCIN""","""Concomitant""","""Concomitant""","""Oral""","""Oral""","""250""","""Milligram""","""Milligramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""4""","""Days""","""Jour(s)""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4282810,173,24440,"""KEFLIN""","""Suspect""","""Suspect""","""Intravenous (not otherwise spe…","""Intraveineuse (sans autre préc…","""1""","""Gram""","""Gramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""3""","""Days""","""Jour(s)""","""POWDER FOR SOLUTION INTRAMUSCU…","""POUDRE POUR SOLUTION INTRAMUSC…"


aqui estan los 

In [26]:
# if report_drugs contains any of the drugnames in interest_drugnames:
reports_with_cephs = report_drug.filter(pl.col("DRUG_PRODUCT_ID").is_in(interest_drugnames["DRUG_PRODUCT_ID"]))
reports_with_cephs = reports_with_cephs.sort("REPORT_ID")
reports_with_cephs

C:\Users\herie\AppData\Local\Temp\ipykernel_15704\3633973611.py:2: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  reports_with_cephs = report_drug.filter(pl.col("DRUG_PRODUCT_ID").is_in(interest_drugnames["DRUG_PRODUCT_ID"]))


REPORT_DRUG_ID,REPORT_ID,DRUG_PRODUCT_ID,DRUGNAME,DRUGINVOLV_ENG,DRUGINVOLV_FR,ROUTEADMIN_ENG,ROUTEADMIN_FR,UNIT_DOSE_QTY,DOSE_UNIT_ENG,DOSE_UNIT_FR,FREQUENCY,FREQ_TIME,FREQUENCY_TIME_ENG,FREQUENCY_TIME_FR,FREQ_TIME_UNIT_ENG,FREQ_TIME_UNIT_FR,THERAPY_DURATION,THERAPY_DURATION_UNIT_ENG,THERAPY_DURATION_UNIT_FR,DOSAGEFORM_ENG,DOSAGEFORM_FR
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
4419404,164,24440,"""KEFLIN""","""Suspect""","""Suspect""","""""","""""","""1""","""Gram""","""Gramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""""","""""","""""","""POWDER FOR SOLUTION INTRAMUSCU…","""POUDRE POUR SOLUTION INTRAMUSC…"
4282810,173,24440,"""KEFLIN""","""Suspect""","""Suspect""","""Intravenous (not otherwise spe…","""Intraveineuse (sans autre préc…","""1""","""Gram""","""Gramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""3""","""Days""","""Jour(s)""","""POWDER FOR SOLUTION INTRAMUSCU…","""POUDRE POUR SOLUTION INTRAMUSC…"
4345176,180,24492,"""KEFLEX""","""Suspect""","""Suspect""","""Oral""","""Oral""","""500""","""Milligram""","""Milligramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""3""","""Days""","""Jour(s)""","""Tablets""","""Comprimés"""
4276502,346,29750,"""CEPHALOTHIN SODIUM""","""Suspect""","""Suspect""","""""","""""","""3.5""","""Gram""","""Gramme""","""""","""""","""""","""""","""""","""""","""12""","""Days""","""Jour(s)""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
339315,356,29752,"""CEPORAN""","""Suspect""","""Suspect""","""Intramuscular""","""Intramusculaire""","""500""","""Drops""","""Gouttes""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
5221398,908140706,58275,"""CEPHALEXIN""","""Concomitant""","""Concomitant""","""""","""""","""2""","""Dosage forms""","""Formes de dosage""","""1""",""".3""","""Days""","""Jour(s)""","""1 every .3 Days""","""1 chaque .3 Jour(s)""","""""","""""","""""","""Tablet""","""Comprimé"""
5230387,908142798,32968,"""CEFUROXIME AXETIL""","""Suspect""","""Suspect""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4957817,908147666,32514,"""CEFTRIAXONE SODIUM FOR INJECTI…","""Suspect""","""Suspect""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""


In [27]:
# get the unique report ids
ids = reports_with_cephs.select("REPORT_ID").unique().sort("REPORT_ID")
ids

REPORT_ID
i64
164
173
180
346
356
…
908140706
908142798
908147666


In [28]:
report_drug

REPORT_DRUG_ID,REPORT_ID,DRUG_PRODUCT_ID,DRUGNAME,DRUGINVOLV_ENG,DRUGINVOLV_FR,ROUTEADMIN_ENG,ROUTEADMIN_FR,UNIT_DOSE_QTY,DOSE_UNIT_ENG,DOSE_UNIT_FR,FREQUENCY,FREQ_TIME,FREQUENCY_TIME_ENG,FREQUENCY_TIME_FR,FREQ_TIME_UNIT_ENG,FREQ_TIME_UNIT_FR,THERAPY_DURATION,THERAPY_DURATION_UNIT_ENG,THERAPY_DURATION_UNIT_FR,DOSAGEFORM_ENG,DOSAGEFORM_FR
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
265,136627,30441,"""MINOCYCLINE""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
266,136627,33686,"""PULMICORT""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
267,166980,33571,"""VASOTEC""","""Concomitant""","""Concomitant""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
269,13557,34527,"""HYDROCORTISONE""","""Suspect""","""Suspect""","""Rectal""","""Rectale""","""100""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""7""","""Days""","""Jour(s)""","""Enema""","""Lavement"""
271,13557,32824,"""PREDNISONE""","""Suspect""","""Suspect""","""Oral""","""Oral""","""40""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""14""","""Days""","""Jour(s)""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4399335,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Oral""","""Oral""","""5""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399342,907360553,35583,"""XELJANZ""","""Suspect""","""Suspect""","""Unknown""","""Inconnu""","""4""","""Milligram""","""Milligramme""","""1""","""1""","""Days""","""Jour(s)""","""1 every 1 Days""","""1 chaque 1 Jour(s)""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4399348,907360553,34916,"""OTEZLA""","""Suspect""","""Suspect""","""Oral""","""Oral""","""3""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""


In [29]:
# keep report_drugs where REPORT_ID is in ids
report_drug_filtered = report_drug.filter(pl.col("REPORT_ID").is_in(ids["REPORT_ID"]))
report_drug_filtered = report_drug_filtered.sort("REPORT_ID")
report_drug_filtered

C:\Users\herie\AppData\Local\Temp\ipykernel_15704\831251734.py:2: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  report_drug_filtered = report_drug.filter(pl.col("REPORT_ID").is_in(ids["REPORT_ID"]))


REPORT_DRUG_ID,REPORT_ID,DRUG_PRODUCT_ID,DRUGNAME,DRUGINVOLV_ENG,DRUGINVOLV_FR,ROUTEADMIN_ENG,ROUTEADMIN_FR,UNIT_DOSE_QTY,DOSE_UNIT_ENG,DOSE_UNIT_FR,FREQUENCY,FREQ_TIME,FREQUENCY_TIME_ENG,FREQUENCY_TIME_FR,FREQ_TIME_UNIT_ENG,FREQ_TIME_UNIT_FR,THERAPY_DURATION,THERAPY_DURATION_UNIT_ENG,THERAPY_DURATION_UNIT_FR,DOSAGEFORM_ENG,DOSAGEFORM_FR
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
4419402,164,3503,"""LARGACTIL 50""","""Concomitant""","""Concomitant""","""Intramuscular""","""Intramusculaire""","""50""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""1""","""Days""","""Jour(s)""","""SOLUTION INTRAMUSCULAR""","""SOLUTION POUR ADMINISTRATION I…"
4419405,164,32842,"""LIBRIUM""","""Concomitant""","""Concomitant""","""Oral""","""Oral""","""10""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""
4419403,164,656,"""LASIX PARENTERAL INJ""","""Concomitant""","""Concomitant""","""Intravenous (not otherwise spe…","""Intraveineuse (sans autre préc…","""40""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""LIQUID INTRAMUSCULAR""","""LIQUIDE POUR ADMINISTRATION IN…"
4419406,164,1625,"""APRESOLINE INJECTION 20MG/AMPO…","""Concomitant""","""Concomitant""","""Intramuscular""","""Intramusculaire""","""20""","""Milligram""","""Milligramme""","""""","""""","""""","""""","""""","""""","""1""","""Days""","""Jour(s)""","""POWDER FOR SOLUTION INTRAVENOU…","""POUDRE POUR SOLUTION INTRAVEIN…"
4419404,164,24440,"""KEFLIN""","""Suspect""","""Suspect""","""""","""""","""1""","""Gram""","""Gramme""","""4""","""1""","""Days""","""Jour(s)""","""4 every 1 Days""","""4 chaque 1 Jour(s)""","""""","""""","""""","""POWDER FOR SOLUTION INTRAMUSCU…","""POUDRE POUR SOLUTION INTRAMUSC…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4949073,908149644,58275,"""CEPHALEXIN""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""250""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""""",""""""
4949077,908149644,31567,"""MYRBETRIQ""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""50""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""TABLET (EXTENDED-RELEASE)""","""COMPRIMÉ (LIBÉRATION PROLONGÉE…"
4949078,908149644,33553,"""ACETYLSALICYLIC ACID""","""Concomitant""","""Concomitant""","""Unknown""","""Inconnu""","""81""","""Milligram""","""Milligramme""","""1""","""""","""""","""""","""""","""""","""""","""""","""""","""NOT SPECIFIED""","""NON PRÉCISÉE"""


In [30]:
# save as parquet
report_drug_filtered.write_parquet(OUTPUT + "report_drug.parquet")

## Report Links

In [ ]:
rl = pl.read_csv(
    PATH + "report_links.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
rl

column_1,column_2,column_3,column_4,column_5
i64,i64,str,str,str
1625021,162502,"""Linked""","""Liè""","""000162500"""
1625841,162584,"""Duplicate""","""Doublon""","""E2B_02161285"""
1626651,162665,"""Duplicate""","""Doublon""","""000302420"""
1630321,163032,"""Duplicate""","""Doublon""","""E2B_00201335"""
1630711,163071,"""Duplicate""","""Doublon""","""E2B_02186057"""
…,…,…,…,…
9076841981,907684198,"""Linked""","""Liè""","""E2B_07690518"""
9076842201,907684220,"""Linked""","""Liè""",""""""
9076842202,907684220,"""Linked""","""Liè""",""""""


In [ ]:
rl = rl.rename({
    "column_1":  "REPORT_LINK_ID",
    "column_2":  "REPORT_ID",
    "column_3":  "RECORD_TYPE_ENG",
    "column_4":  "RECORD_TYPE_FR",
    "column_5":  "REPORT_LINK_NO"
})
rl.write_parquet(OUTPUT + "report_links.parquet")

## Report Type

In [ ]:
rt = pl.read_csv(
    PATH + "report_type_lx.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
rt

column_1,column_2,column_3,column_4
i64,i64,str,str
70095001,5001,"""CMDSNet""","""ResSCMM"""
70095002,5002,"""Investigational Testing""","""Essai expérimental"""
70095007,5,"""Published""","""Publiée"""
70095009,7,"""Spontaneous""","""Déclaration spontanée"""
70095010,8,"""Study""","""Étude"""
…,…,…,…
70095021,5003,"""Industry 10-day""","""Industrie (10 jours)"""
70095022,5004,"""Industry 30-day""","""Industrie (30 jours)"""
70095023,5005,"""Industry Foreign""","""Industrie à l'étranger"""


In [ ]:
rt = rt.rename({
    "column_1":  "REPORT_TYPE_LX_ID",
    "column_2":  "REPORT_TYPE_CODE",
    "column_3":  "REPORT_TYPE_EN",
    "column_4":  "REPORT_TYPE_FR"
})
rt.write_parquet(OUTPUT + "report_type_lx.parquet")

## Reports

In [ ]:
import polars as pl
PATH = "data/Canada Vigilance Adverse Reaction Online Database/"

reports = pl.read_csv(
    PATH + "reports.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
reports

column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42
i64,i64,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,1,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""62""","""62""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""66""","""Kilogram""","""Kilogramme""","""155""","""Centimeter""","""Centimètres""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""Physician""","""Physicien""","""""","""""","""""","""""","""""",""""""
2,2,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""70""","""70""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
3,3,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""1""","""Male""","""Masculin""","""83""","""83""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""44""","""Kilogram""","""Kilogramme""","""173""","""Centimeter""","""Centimètres""",1,"""Serious""","""Grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
4,4,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""78""","""78""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
5,5,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""55""","""55""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""""","""""","""01""","""Community""","""Communauté""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
908151570,null,0,"""30-JUN-25""","""30-JUN-25""","""202501618""",8,"""Study""","""Étude""","""1""","""Male""","""Masculin""","""75""","""75""","""Years""","""Années""",11,"""Fatal""","""Fatale""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""1""","""2""","""2""","""2""","""2""","""1""","""Other health professional""","""Autre professionnel de santé""","""07""","""MAH""","""Fabricant""","""CA-HLS-202501618""","""""","""CA-HLS-202501618"""
908151575,null,0,"""30-JUN-25""","""30-JUN-25""","""2025033697""",8,"""Study""","""Étude""","""2""","""Female""","""Féminin""","""42""","""42""","""Years""","""Années""",9,"""Not recovered/not resolved""","""Non rétabli/Non résolu""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""2""","""2""","""2""","""2""","""2""","""1""","""""","""""","""07""","""MAH""","""Fabricant""","""CA-UCBSA-2025033697""","""""","""CA-UCBSA-2025033697"""
908151595,null,0,"""30-JUN-25""","""30-JUN-25""","""2025TUS040335""",8,"""Study""","""Étude""","""2""","""Female""","""Féminin""","""22""","""22""","""Years""","""Années""",8,"""Recovering/resolving""","""Guérison en cours""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""2""","""2""","""2""","""2""","""1""","""1""","""Other heal

In [ ]:
reports = reports.rename({
    "column_1":  "REPORT_ID",
    "column_2":  "REPORT_NO",
    "column_3":  "VERSION_NO",
    "column_4":  "DATRECEIVED",
    "column_5":  "DATINTRECEIVED",
    "column_6":  "MAH_NO",
    "column_7":  "REPORT_TYPE_CODE",
    "column_8":  "REPORT_TYPE_ENG",
    "column_9":  "REPORT_TYPE_FR",
    "column_10": "GENDER_CODE",
    "column_11": "GENDER_ENG",
    "column_12": "GENDER_FR",
    "column_13": "AGE",
    "column_14": "AGE_Y",
    "column_15": "AGE_UNIT_ENG",
    "column_16": "AGE_UNIT_FR",
    "column_17": "OUTCOME_CODE",
    "column_18": "OUTCOME_ENG",
    "column_19": "OUTCOME_FR",
    "column_20": "WEIGHT",
    "column_21": "WEIGHT_UNIT_ENG",
    "column_22": "WEIGHT_UNIT_FR",
    "column_23": "HEIGHT",
    "column_24": "HEIGHT_UNIT_ENG",
    "column_25": "HEIGHT_UNIT_FR",
    "column_26": "SERIOUSNESS_CODE",
    "column_27": "SERIOUSNESS_ENG",
    "column_28": "SERIOUSNESS_FR",
    "column_29": "DEATH",
    "column_30": "DISABILITY",
    "column_31": "CONGENITAL_ANOMALY",
    "column_32": "LIFE_THREATENING",
    "column_33": "HOSP_REQUIRED",
    "column_34": "OTHER_MEDICALLY_IMP_COND",
    "column_35": "REPORTER_TYPE_ENG",
    "column_36": "REPORTER_TYPE_FR",
    "column_37": "SOURCE_CODE",
    "column_38": "SOURCE_ENG",
    "column_39": "SOURCE_FR",
    "column_40": "E2B_IMP_SAFETYREPORT_ID",
    "column_41": "AUTHORITY_NUMB",
    "column_42": "COMPANY_NUMB"
})

In [ ]:
reports

REPORT_ID,REPORT_NO,VERSION_NO,DATRECEIVED,DATINTRECEIVED,MAH_NO,REPORT_TYPE_CODE,REPORT_TYPE_ENG,REPORT_TYPE_FR,GENDER_CODE,GENDER_ENG,GENDER_FR,AGE,AGE_Y,AGE_UNIT_ENG,AGE_UNIT_FR,OUTCOME_CODE,OUTCOME_ENG,OUTCOME_FR,WEIGHT,WEIGHT_UNIT_ENG,WEIGHT_UNIT_FR,HEIGHT,HEIGHT_UNIT_ENG,HEIGHT_UNIT_FR,SERIOUSNESS_CODE,SERIOUSNESS_ENG,SERIOUSNESS_FR,DEATH,DISABILITY,CONGENITAL_ANOMALY,LIFE_THREATENING,HOSP_REQUIRED,OTHER_MEDICALLY_IMP_COND,REPORTER_TYPE_ENG,REPORTER_TYPE_FR,SOURCE_CODE,SOURCE_ENG,SOURCE_FR,E2B_IMP_SAFETYREPORT_ID,AUTHORITY_NUMB,COMPANY_NUMB
i64,i64,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,1,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""62""","""62""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""66""","""Kilogram""","""Kilogramme""","""155""","""Centimeter""","""Centimètres""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""Physician""","""Physicien""","""""","""""","""""","""""","""""",""""""
2,2,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""70""","""70""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
3,3,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""1""","""Male""","""Masculin""","""83""","""83""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""44""","""Kilogram""","""Kilogramme""","""173""","""Centimeter""","""Centimètres""",1,"""Serious""","""Grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
4,4,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""78""","""78""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""""","""""","""""","""""","""""","""""","""""","""""","""03""","""Hospital""","""Hôpital""","""""","""""",""""""
5,5,0,"""05-JUN-73""","""05-JUN-73""","""""",7,"""Spontaneous""","""Déclaration spontanée""","""2""","""Female""","""Féminin""","""55""","""55""","""Years""","""Années""",6,"""Unknown""","""Inconnue""","""""","""""","""""","""""","""""","""""",2,"""Not Serious""","""Non grave""","""""","""""","""""","""""","""""","""""","""""","""""","""01""","""Community""","""Communauté""","""""","""""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
908151570,null,0,"""30-JUN-25""","""30-JUN-25""","""202501618""",8,"""Study""","""Étude""","""1""","""Male""","""Masculin""","""75""","""75""","""Years""","""Années""",11,"""Fatal""","""Fatale""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""1""","""2""","""2""","""2""","""2""","""1""","""Other health professional""","""Autre professionnel de santé""","""07""","""MAH""","""Fabricant""","""CA-HLS-202501618""","""""","""CA-HLS-202501618"""
908151575,null,0,"""30-JUN-25""","""30-JUN-25""","""2025033697""",8,"""Study""","""Étude""","""2""","""Female""","""Féminin""","""42""","""42""","""Years""","""Années""",9,"""Not recovered/not resolved""","""Non rétabli/Non résolu""","""""","""""","""""","""""","""""","""""",1,"""Serious""","""Grave""","""2""","""2""","""2""","""2""","""2""","""1""","""""","""""","""07""","""MAH""","""Fabricant""","""CA-UCBSA-2025033697""","""""","""CA-UCBSA-2025033697"""
908151595,null,0,"""30-JUN-25""","""30-JUN-25""","""2025TUS040335""",8,"""Study""","""Étude""","""2""","""Female""","""Féminin""","""22""","""22""","""Years""","""Années""",8,"""Recovering/resolving""","""Guérison e

In [ ]:
reports['OTHER_MEDICALLY_IMP_COND'].unique()

OTHER_MEDICALLY_IMP_COND
str
""""""
"""1"""
"""2"""
"""0"""


In [ ]:
# save as parquet
reports.write_parquet(OUTPUT + "reports.parquet")

## Seriousness

In [ ]:
ser = pl.read_csv(
    PATH + "seriousness_lx.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
ser

column_1,column_2,column_3,column_4
i64,i64,str,str
2001,1,"""Serious""","""Grave"""
2002,2,"""Not Serious""","""Non grave"""


In [ ]:
ser = ser.rename({
    "column_1":  "SERIOUSNESS_LX_ID",
    "column_2":  "SERIOUSNESS_CODE",
    "column_3":  "SERIOUSNESS_EN",
    "column_4":  "SERIOUSNESS_FR"
})
ser.write_parquet(OUTPUT + "seriousness_lx.parquet")

## Source

In [ ]:
sou = pl.read_csv(
    PATH + "source_lx.txt",
    separator="$",
    quote_char='"',
    has_header=False,
    ignore_errors=True
)
sou

column_1,column_2,column_3,column_4
i64,i64,str,str
1001,1,"""Community""","""Communauté"""
1002,2,"""Clinical Study""","""Essai Clinique"""
1003,3,"""Hospital""","""Hôpital"""
1004,4,"""Distributor""","""Distributeur"""
1007,7,"""MAH""","""Fabricant"""
…,…,…,…
1022,22,"""Unknown""","""Inconnu"""
105001,5001,"""Importer""","""importateur"""
105002,5002,"""Manufacturer""","""fabricant"""


In [ ]:
sou = sou.rename({
    "column_1":  "SOURCE_LX_ID",
    "column_2":  "SOURCE_CODE",
    "column_3":  "SOURCE_EN",
    "column_4":  "SOURCE_FR"
})  
sou.write_parquet(OUTPUT + "source_lx.parquet")